In [ ]:
import geopandas as gpd
import numpy as np

from osgeo import gdal, ogr, osr
from gdalconst import *
gdal.UseExceptions()

import rasterio

import stac

## Obter imagens da área de interesse via STAC (BrazilDataCube)

In [ ]:
# Criar conexão com o servidor da STAC
bdc_stac_service = stac.STAC('http://brazildatacube.dpi.inpe.br/')

# Obter itens filtrados pelo objeto JSON (https://stacspec.org/STAC-api.html#operation/postSearchSTAC)
# Filtro: 200 primeiras imagens da coleção 'LC8SR-1' que interserctam o ponto (-46.872, -17.150).
item = bdc_stac_service.search({'collections':['LC8SR-1'], 
                                "intersects": {"type": "Point",
                                               "coordinates": [-46.872, -17.150]},
                                'limit': 200})

# Mostrar quantas imagens foram recuperadas
len(item.features)

## Criar um dicionário com as datas e links das imagens

In [ ]:
links = {}

# Percorrer todos o itens obtidos ds STAC
for i in item.features:
    
    # Adquirir a data da imagem
    date = i['properties']['datetime'][0:10]
    
    # Inserir um item no dicionário 'links' com (chave = data da imagem) e (valor = URL da imagem).
    # IV selecionado: NDVI
    links[date] = i['assets']['sr_ndvi']['href']

# Mostrar dicionário de links
links

## Abrir arquivo de pontos e extrair coordenadas

In [ ]:
# Abrir arquivo de pontos
df = gpd.read_file('./DADOS/pt_inicial_10qtd.geojson')

# Gerar duas listas (lat e lon) com as coordenadas dos pontos
lat = []
lon = []
for i in df.geometry:
    lon.append(i.x)
    lat.append(i.y)

## Amostrar os valores das imagens para os pontos obtidos

A célula abaixo amostra os valores de todas as imagens (93) para todos o pontos (1094). Para isso ela precisa abrir cada uma como dataset e processá-la a partir de todos os pontos definidos (93 x 1094). 

A sua execução demanda mais tempo que as demais (aprx. 20 min).

In [ ]:
# Criar um dicionário de saída
out = {}

# Percorrer todas os pares data(k)/URL(v) da lista 'links'
for k,v in links.items():
    # Abrir a imagem da URL
    with rasterio.open(v) as src:
        # Obter o valores dos pixel da imagem para as coordenadas dos pontos (amostrar valores)
        val = [x for x in src.sample(zip(lon,lat))]
        
        # Transformar cada item (array unitario) da lista 'val' para número
        for i in range(len(val)):
            val[i] = val[i][0]
            
        # Inserir o item no dicionário 'out' com (chave = data da imagem) e (valor = lista de valores da série).
        out[k] = val

In [ ]:
# Inserir os valores obtidos como colunas no dataframe dos pontos
for k,v in out.items():
    df.insert(loc=2,column=k,value=v)

# Agrupar os pontos de cada pivo a partir da média (e excluir a coluna 'id')
df = df.groupby(['PIVOID']).mean().drop('id',1)

In [ ]:
df

## Selecionar 6 pivôs aleatoriamente e plotar as suas respectivas séries

A cada execução da célula abaixo são sorteados 6 pivos diferentes

In [ ]:
# Selecionar 6 pivôs aleatoriamente, transpor o dataframe e converter o valores para o intervalo [0,1] (divisão por 10000)
df_reduzido = df.sample(6).T/10000
# Plotar as séries
df_reduzido.plot(subplots=True, figsize=(15,10));